<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/federated_learning_project_remote_arithmetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project IX
## So, it's time to get started with PySyft and Federated Learning

The idea of federated learning is that in order for machines to learn about data , the data now need not move from data to some kind of central server but now a model made at server can now go to local devices (remote machines) and then train the model to their local dataset and then update the model and send it back to the server.
This allows us to save the data from leakage , save bandwidth , more improved models as more  training examples as millions of devices (remotly) connect with servers.

Let's first install syft.

In [0]:
pip install syft

In [0]:
import torch as th
import syft as sy

Let's now create hook which modifies pysyft for pytorch and gives all its functionality to be used with pytorch.

In [53]:
hook = sy.TorchHook(th)

W0706 08:25:38.096700 140272010098560 hook.py:98] Torch was already hooked... skipping hooking process


so, to connect and to be able to do operations on remote tensors located on remoted machines, to get and set them , we need some kind of interface to connect our server with the local remote device/machine.
There comes , concept of worker , which facilitates this interface and allows us to perform all the operations on remote tensors.
Let's create them and play around with them.
whatever we will get and send all will work with and through pointers.

In [0]:
bob = sy.VirtualWorker(hook, id = "bob")  # this creates our first worker named id as bob



In [55]:
bob._objects

{}

In [0]:
x = th.tensor([1, 2, 3, 4, 5]) # let's create the tensor required to be sent to our worker - "bob"
x = x.send(bob) # sending to bob

In [57]:
x # pointer from my machine to bob 

(Wrapper)>[PointerTensor | me:51685196821 -> bob:97523591780]

In [58]:
bob._objects # now bob has those tensors

{97523591780: tensor([1, 2, 3, 4, 5])}

Now , x has two meta data - its location and id at location which identifies the worker

In [59]:
x.location

<VirtualWorker id:bob #objects:1>

In [60]:
x.id_at_location


97523591780

In [61]:
x.id

51685196821

In [62]:
x.owner # by default we are the owner of this tensor x , it means everytime we hook torch to pysyft , there is already a virtual worker created at that time


<VirtualWorker id:me #objects:0>

In [63]:
hook.local_worker


<VirtualWorker id:me #objects:0>

In [64]:
x.get() # now we are taking back the tensor from bob

tensor([1, 2, 3, 4, 5])

In [65]:
bob._objects # now bob has nothing !

{}

Let's create two virtual workers ! bob and alice - 

In [0]:
bob = sy.VirtualWorker(hook, id = "bob") # first worker
alice = sy.VirtualWorker(hook, id = "alice") # second worker
x = th.tensor([1, 2, 3, 4, 5])

In [0]:
x = x.send(bob, alice) # sending multiple workers at a time 

In [76]:
x # multipoint tensor which points to different machines

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:53459958730 -> bob:56727341380]
	-> (Wrapper)>[PointerTensor | me:64200850796 -> alice:80927549445]

In [77]:
bob._objects # checking bob objects

{11232225584: tensor([]), 56727341380: tensor([1, 2, 3, 4, 5])}

In [78]:
alice._objects # checking alice objects

{72669233561: tensor([]), 80927549445: tensor([1, 2, 3, 4, 5])}

In [79]:
x.get() # getting both tensors at a time

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [80]:
bob._objects # now objects should not contain tensors

{11232225584: tensor([])}

In [82]:
alice._objects # similar for this 

{72669233561: tensor([])}

In [0]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice) 

In [0]:
z = x.get(sum_results = True)

In [91]:
z

tensor([ 2,  4,  6,  8, 10])

Let's do some remote aritmetic on tensors

In [0]:
# Two different tensors send to bob as if we will send two tensors to different workers , we can't do operations on both of them 
x = th.tensor([1,2,3,4,5]).send(bob) 
y = th.tensor([1,1,1,1,1]).send(bob)

In [94]:
x

(Wrapper)>[PointerTensor | me:51583300043 -> bob:22761749523]

In [95]:
y

(Wrapper)>[PointerTensor | me:35015651872 -> bob:64525715631]

In [0]:
z = x + y

In [97]:
z

(Wrapper)>[PointerTensor | me:12617089070 -> bob:66469428747]

In [98]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

Simple backpropogation steps in pysyft

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [105]:
z = (x + y).sum()
z.backward()


(Wrapper)>[PointerTensor | me:51984582036 -> bob:33746314282]

In [106]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [107]:
x.grad


tensor([1., 1., 1., 1., 1.])

In [0]:
y = y.get()

In [109]:
y

tensor([1., 1., 1., 1., 1.], requires_grad=True)

In [110]:
y.grad

tensor([1., 1., 1., 1., 1.])

In [113]:
z

(Wrapper)>[PointerTensor | me:99126214757 -> bob:1206692804]